# Chapter 12. k-Nearest Neighbors

In [3]:
from __future__ import division
from collections import Counter
from linear_algebra import distance
from statistics import mean
import math, random
import matplotlib.pyplot as plt

Imagine that you're trying to predict how I'm going to vote in the next presidential election.  
If you know nothing else about me, one sensible approach is to look at how my *neighbors* are planning to vote.  
Now imagine that you know more about me than just geography -- my age, my income, marital status, and so on.  
To the extent that my behavior is influenced (or characterized) by those things, looking just at my neighbors who are close to me among all of those dimensions seems likely to be an even better predictor than looking at all of my neighbors.  
This is the idea behind [nearest neighbors classification](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm).

## The Model